In [1]:
import time
import warnings

import numpy as np
from numpy.linalg import norm

import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib.pyplot import Figure, Axes

from typing import List

from sklearn.model_selection import RepeatedKFold

from pyROMs.pod import POD
from readers import NeutronicsDatasetReader

from utils import *

warnings.filterwarnings("ignore")

Define Routines

In [2]:
def cross_validation(dataset: "NeutronicsDatasetReader", pod: "POD",
                     n_splits: int = 5, n_repeats: int = 100,
                     interior_only: bool = False, vars: str = None):
    """
    Cross-validation study.

    Parameters
    ----------
    dataset : NeutronicsDatasetReader
        The dataset reader containing the data.
    pod : POD
        The POD model with the appropriate hyper-parameters.
    n_splits : int, default 5
        The number of splits for the cross-validation study.
    n_repeats : int, defualt 100
        The number of k-fold repeats.
    interior_only : bool, default False
        A flag to include only interior values in the validation set.
    vars : str, default None
        The variables to include in the study.
    """
    # Generate data
    X = dataset.create_dataset_matrix(vars)
    Y = dataset.parameters

    # Storage
    means, maxs, mins = [], [], []
    construction, query = [], []

    # Define cross-validator
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats)
    if interior_only:
        interior = dataset.interior_map
        iterator = cv.split(X[interior], Y[interior])
    else:
        iterator = cv.split(X, Y)

    # Start cross-validations
    for train, test in iterator:
        if interior_only:
            X_train, Y_train = X[interior][train], Y[interior][train]
            X_test, Y_test = X[interior][test], Y[interior][test]
            X_train = np.vstack((X_train, X[dataset.boundary_map]))
            Y_train = np.vstack((Y_train, Y[dataset.boundary_map]))
        else:
            X_train, Y_train = X[train], Y[train]
            X_test, Y_test = X[test], Y[test]

        # Construct ROM
        t_start = time.time()
        pod.fit(X_train, Y_train)
        t_end = time.time()
        construction.append(t_end - t_start)

        # Make predictions
        t_start = time.time()
        X_pod = pod.predict(Y_test)
        t_end = time.time()
        query.append(t_end - t_start)

        # Compute errors
        errors = np.zeros(len(X_pod))
        for i in range(len(X_pod)):
            errors[i] = norm(X_test[i] - X_pod[i]) / norm(X_test[i])
        means.append(np.mean(errors))
        maxs.append(np.max(errors))
        mins.append(np.min(errors))

    print()
    print(f"{'Number of POD Modes:':<30}\t{pod.n_modes}")
    print(f"{'Number of Snapshots:':<30}\t{pod.n_snapshots}")
    print(f"{'Number of Validations:':<30}\t{len(X_pod)}")
    print(f"{'Avg Construction Time:':<30}\t{np.mean(construction):.3e} s")
    print(f"{'Avg Query Time':<30}\t{np.mean(query):.3e} s")
    print_statistics(means, maxs, mins)
    return means, maxs, mins


def print_statistics(means, maxs, mins):
    """
    Print the statistics corresponding to the error distributions.

    Parameters
    ----------
    means : list of float
    maxs : list of float
    mins : list of float
    """

    mean_95conf = np.percentile(means, [2.5, 97.5])
    max_95conf = np.percentile(maxs, [2.5, 97.5])
    min_95conf = np.percentile(mins, [2.5, 97.5])

    print()
    print(f"Mean Error Statistics:")
    print(f"\tMean:\t{np.mean(means):.3e}")
    print(f"\tMedian:\t{np.median(means):.3e}")
    print(f"\t95% CI:\t[{mean_95conf[0]:.3e}, {mean_95conf[1]:.3e}]")
    print("\nMaximum Error Statistics:")
    print(f"\tMean:\t{np.mean(maxs):.3e}")
    print(f"\tMedian:\t{np.median(maxs):.3e}")
    print(f"\t95% CI:\t[{max_95conf[0]:.3e}, {max_95conf[1]:.3e}]")
    print("\nMinimum Error Statistics:")
    print(f"\tMean:\t{np.mean(mins):.3e}")
    print(f"\tMedian:\t{np.median(mins):.3e}")
    print(f"\t95% CI:\t[{min_95conf[0]:.3e}, {min_95conf[1]:.3e}]")


def plot_distributions(means, maxs):
    """
    Plot the mean and max error distributions.

    Parameters
    ----------
    means : list of float
    maxs : list of float
    """
    fig: Figure = plt.figure()
    for i in range(2):
        data = means if i == 0 else maxs
        xlabel = "Mean" if i == 0 else "Maximum"
        ylabel = "Probability" if i == 0 else ""

        ax: Axes = fig.add_subplot(1, 2, i + 1)
        ax.tick_params(labelsize=12)
        ax.set_xlabel(f"{xlabel} Error", fontsize=14)
        ax.set_ylabel(ylabel, fontsize=14)
        ax.grid(True)
        sb.histplot(data, bins=20, stat='probability', kde=True,
                    log_scale=True, ax=ax)
    plt.tight_layout()


def print_general_table(means, maxs, mins):
    """
    Print a LaTeX table describing general information about the
    cross-validation study.

    Parameters
    ----------
    means : list of float
    maxs : list of float
    """
    msg = "\\begin{tabular}{|c|c|}"
    msg += "\n\t\hline"
    msg += "\n\t\\textbf{Quantity} & \\textbf{Value} \\\\ \hline\hline"
    msg += f"\n\tMean of Set Means & {np.mean(means):.3e} \\\\ \hline"
    msg += f"\n\tMaximum of Set Means & {np.max(means):.3e} \\\\ \hline"
    msg += f"\n\tMinimum of Set Means & {np.min(means):.3e} \\\\ \hline"
    msg += f"\n\tMean of Set Maximums & {np.mean(maxs):.3e} \\\\ \hline"
    msg += f"\n\tMaximum of Set Maximums & {np.max(maxs):.3e} \\\\ \hline"
    msg += f"\n\tMean of Set Minimums & {np.mean(mins):.3e} \\\\ \hline"
    msg += f"\n\tMinimum of Set Minimums & {np.min(mins):.3e} \\\\ \hline"
    msg += f"\n\end{{tabular}}"
    print(msg)


def print_statistics_table(means, maxs):
    """
    Print a LaTeX table.

    Parameters
    ----------
    means : list of float
    maxs : list of float
    """
    conf_mean = np.percentile(means, [2.5, 97.5])
    conf_max = np.percentile(maxs, [2.5, 97.5])

    msg = "\\begin{tabular}{|c|c|c|}"
    msg += "\n\t\hline \\textbf{Quantity} & \\textbf{Mean Error} & " \
           "\\textbf{Max Error} \\\\ \hline"
    msg += f"\n\tMean & {np.mean(means):.3e} & " \
           f"{np.mean(maxs):.3e} \\\\ \hline"
    msg += f"\n\tStd. Deviation & {np.std(means):.3e} & " \
           f"{np.std(maxs):.3e} \\\\ \hline"
    msg += f"\n\tMedian & {np.median(means):.3e} & " \
           f"{np.median(maxs):.3e} \\\\ \hline"
    msg += f"\n\t95\% Conf. Interval & [{conf_mean[0]:.3e}, {conf_mean[1]:.3e}] & " \
           f"[{conf_max[0]:.3e}, {conf_max[1]:.3e}] \\\\ \hline"
    msg += f"\n\end{{tabular}}"
    print()
    print(msg)

Parse the Data

In [3]:
problem_name = input("What problem? ")

print("Loading and formating the data...")
t_start = time.time()
dataset = get_data(problem_name)
t_end = time.time()
print(f"Loading the data took {t_end - t_start:3f} s")

Loading and formating the data...
Loading the data took 13.387334 s


KFold Cross Validation

In [4]:
n_splits = 5
n_repeats = 500 // n_splits
interior_only = False

params = get_params(problem_name)
vars = params["vars"]
svd_rank = 1.0 - params["tau"]
interp = params["interp"]
epsilon = params["epsilon"]

pod = POD(svd_rank=svd_rank,
          interpolation_method=interp,
          epsilon=epsilon)

res = cross_validation(dataset, pod, n_splits,
                       n_repeats, interior_only, vars)
means, maxs, mins = res

TypeError: __init__() got an unexpected keyword argument 'interpolation_method'

In [ ]:
plot_distributions(means, maxs)
print_statistics_table(means, maxs)
